In [0]:
from google.colab import drive
import networkx as nx
import pandas as pd
import numpy as np
import os
import random
from tqdm import tqdm
!pip install stellargraph
import stellargraph as sg
from stellargraph.data import UnsupervisedSampler
from stellargraph.mapper import Attri2VecLinkGenerator, Attri2VecNodeGenerator
from stellargraph.layer import Attri2Vec, link_classification
from sklearn import decomposition

from tensorflow import keras

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score


     |████████████████████████████████| 440kB 11.0MB/s 
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


### Подгружаем данные и приводим в удобный для использования вид

Коннектимся к гугл-диску с файлами

In [0]:
drive.mount('/content/drive')
%cd /content/drive/My\ Drive/graph

/content/drive/My Drive/graph


Загружаем данные о вершинах и ребрах, связываем дополнительные сведения о вершинах в одну таблицу

In [0]:
%%time
edges = pd.read_csv('./edges.csv')
vertices = pd.read_csv('./vertices_edit.csv', index_col=0) 

CPU times: user 2.85 s, sys: 262 ms, total: 3.11 s
Wall time: 13.5 s


/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [ ]:
vertices['main_okved'] = vertices['main_okved'].astype(str) 
vertices['company_type'] = vertices['company_type'].astype(str) 
vertices['region_code'] = vertices['region_code'].astype(str) 
vertices['main_okved2'] = vertices['main_okved'].apply(lambda x: x.split('.')[0])

In [0]:
regions = pd.read_csv('./regions.csv', index_col=0)
regions['code'] = regions['code'].astype(str)
regions = regions.set_index('code')
regions.head()

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,reg_name,population,fed_district
code,,,
77,Москва,12506468,Центральный
50,Московская область,7503385,Центральный
23,Краснодарский край,5603420,Южный
78,Санкт-Петербург,5351935,Северо-Западный
66,Свердловская область,4325256,Уральский


In [0]:
okved = pd.read_csv('./okved.csv', index_col=0)
okved['code'] = okved['code'].astype(str)
okved.head()

,code,okved,letter
0,1,"СЕЛЬСКОЕ, ЛЕСНОЕ ХОЗЯЙСТВО, ОХОТА, РЫБОЛОВСТВО...",А
1,2,"СЕЛЬСКОЕ, ЛЕСНОЕ ХОЗЯЙСТВО, ОХОТА, РЫБОЛОВСТВО...",А
2,3,"СЕЛЬСКОЕ, ЛЕСНОЕ ХОЗЯЙСТВО, ОХОТА, РЫБОЛОВСТВО...",А
3,5,ДОБЫЧА ПОЛЕЗНЫХ ИСКОПАЕМЫХ,B
4,6,ДОБЫЧА ПОЛЕЗНЫХ ИСКОПАЕМЫХ,B


In [0]:
vertices = vertices.join(okved.set_index('code'), on='main_okved2')
vertices['region_code'] = vertices['region_code'].astype(str)
vertices = vertices.join(regions, on='region_code')
vertices = vertices.drop(['reg_name', 'okved'], axis=1)
vertices['population'] = vertices['population'].apply(np.log)
vertices.head()

,main_okved,region_code,company_type,n_transactions,value,main_okved2,letter,population,fed_district
id,,,,,,,,,
1,46.75,77,Limited,0.698135,13.898125,46,G,12506468.0,Центральный
2,41.2,78,Limited,5.235162,14.052657,41,F,5351935.0,Северо-Западный
3,25.11,50,Limited,5.336584,13.381677,25,С,7503385.0,Центральный
4,45.31,89,Limited,6.404597,14.527384,45,G,538547.0,Уральский
5,56.1,50,Limited,6.283210,14.628678,56,I,7503385.0,Центральный


Приводим данные о ребрах к логарифмированной шкале

In [0]:
min_v = -edges['value'].min()
edges['value'] = edges['value'].apply(lambda x: np.log(x+min_v + 1))
edges = edges.replace(-np.inf, 0)
edges = edges.replace(np.inf, 0)
vertices = vertices.replace(np.nan, 0)
edges = edges.replace(np.nan, 0)

In [0]:
edges['n_transactions'] = edges['n_transactions'].apply(lambda x: np.log(x+1))
edges = edges.replace(-np.inf, 0)
edges = edges.replace(np.inf, 0)
vertices = vertices.replace(np.nan, 0)
edges = edges.replace(np.nan, 0)

Запишем категориальные данные в виде ohe-столбцов.

In [0]:
for column in ['main_okved2', 'company_type', 'letter', 'fed_district']:
    for value in tqdm(vertices[column].unique()):
        vertices[f'{column}_{value}'] = vertices[column].apply(lambda x: int(x == value)).astype('int8')

100%|██████████| 9/9 [00:05<00:00,  1.79it/s]


In [0]:
vertices = vertices.drop(columns=['main_okved', 'region_code', 'main_okved2', 'company_type', 'letter', 'fed_district'])

### Воссоздаем граф:

Уберем из графа вершины, в которые нет ребер

In [0]:
s = set(edges['id_1'].unique()).union(set(edges['id_2'].unique()))
v = set(vertices.index)

vertices_ = vertices.iloc[~vertices.index.isin(list(v.difference(s)))]
vertices_.shape

(1514713, 131)

Сделаем граф networkx из ребер графа с весом, равным числу транзакций

In [0]:
edgelist = edges
edgelist = edgelist.rename(columns = {'id_1':'source', 'id_2':'target'}) # приводим названия колонок к виду, требуемому networkx
G_all_nx = nx.from_pandas_edgelist(edgelist, edge_attr=['n_transactions']) # создаем граф из списка ребер и указываем имя 
                                                                           # атрибута, который будет использован как вес ребра            

Также из полученного графа создадим граф с использованием библиотеки stellargraph. Помимо ребер, теперь граф содержит фичи для вершин.

In [0]:
all_node_features = vertices_
G_all = sg.StellarGraph.from_networkx(G_all_nx, edge_weight_attr='n_transactions', node_features=all_node_features)

In [0]:
print(G_all.info())


StellarGraph: Undirected multigraph
 Nodes: 1514713, Edges: 4685644

 Node types:
  default: [1514713]
    Features: float32 vector, length 131
    Edge types: default-default->default

 Edge types:
    default-default->default: [4685644]
        Weights: range=[0.693147, 6.97849], mean=2.91898, std=2.21072


Пропишем настройки для генератора сэмплов

In [0]:
nodes = list(G_all.nodes())
number_of_walks = 4
length = 3
unsupervised_samples = UnsupervisedSampler(
    G_all, nodes=nodes, length=length, number_of_walks=number_of_walks
)

И для генератора батчей

In [0]:
batch_size = 1024
epochs = 3
generator = Attri2VecLinkGenerator(G_all, batch_size)

link_classification: using 'ip' method to combine node embeddings into edge embeddings


Создаем слои для модели

In [ ]:
layer_sizes = [32] # указываем размер скрытого слоя
attri2vec = Attri2Vec(
    layer_sizes=layer_sizes, generator=generator, bias=False, normalize=None
)
x_inp, x_out = attri2vec.in_out_tensors() # определяем вход и выход attri2vec модели
prediction = link_classification(
    output_dim=1, output_act="sigmoid", edge_embedding_method="ip" # *
)(x_out) # функция для генерации предсказания по эмбеддингам attri2vec модели

\* $ip(u,v) = sum_{i=1..d}{u_i*v_i}$

Собираем все в одну модель

In [0]:
model = keras.Model(inputs=x_inp, outputs=prediction)

model.compile(
    optimizer=keras.optimizers.Adam(lr=1e-2),
    loss=keras.losses.binary_crossentropy,
    metrics=[keras.metrics.binary_accuracy, keras.metrics.Precision()],
)

Генерируем батчи

In [0]:
batches = generator.flow(unsupervised_samples)

Обучаем!

In [0]:
history = model.fit(
    batches,
    epochs=epochs,
    verbose=1,
    shuffle=False,
)

Epoch 1/3
17751/17751 [==============================] - 7607s 429ms/step - loss: 0.5915 - binary_accuracy: 0.6670 - precision: 0.6476
Epoch 2/3
17751/17751 [==============================] - 7425s 418ms/step - loss: 0.5283 - binary_accuracy: 0.7413 - precision: 0.7241
Epoch 3/3
16796/17751 [===========================>..] - ETA: 6:46 - loss: 0.5087 - binary_accuracy: 0.7582 - precision: 0.7481Buffered data was truncated after reaching the output size limit.

В результате получаем точность около 76%. 

### Сохраняем и используем модель

In [0]:
model_json = model.to_json()
with open("model(all,lr1e-2,3).json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("model(all,lr1e-2,3).h5")
print("Saved model to disk")

Saved model to disk


Извлекаем веса из прошлой модели

In [0]:
x_inp_src = x_inp[0]
x_out_src = x_out[0]

Снова определяем генератор, но уже для вершин

In [0]:
batches = Attri2VecNodeGenerator(G_all, batch_size).flow(G_all.nodes())

Делаем модель, которая по фичам выдает векторное представление

In [0]:
model = keras.Model(inputs=x_inp_src, outputs=x_out_src)

Наконец, предсказываем

In [0]:
node_embeddings = model.predict(batches, workers=1, verbose=1)

1480/1480 [==============================] - 3s 2ms/step


Запишем предсказания в датафрейм

In [0]:
df = pd.DataFrame.from_records(node_embeddings)
df.index = G_all.nodes()
df.to_csv("./embeddings(all,lr1e-2,3).csv")

Если необходимо, можно загрузить сохраненную модель

In [0]:
#model.load_weights("model1kk.h5")
#print("Loaded model from disk")